In [ ]:
!pip install langchain-community langchain_openai datasets huggingface_hub

In [ ]:
import os
import re
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from datasets import Dataset
from huggingface_hub import HfApi

In [ ]:
# Google Drive를 Google Colab에 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1단계. PDF 파일에서 JSON 문자열을 추출하여 사전형으로 변환하는 기능

In [ ]:
def extract_and_parse_json(text):
    """
    텍스트에서 JSON 문자열을 추출하여 사전형으로 변환하는 기능
    """
    try:
        match = re.search(r"\{.*\}", text, re.DOTALL)
        json_string = match.group() if match else ""
        return json.loads(json_string)
    except (AttributeError, json.JSONDecodeError):
        return {}

## 2단계. PDF 파일을 JSON으로 변환

보안 매뉴얼 PDF 파일을 로드하고, 질문-답변 형식으로 변환합니다.
이 데이터를 JSON 형식으로 추출하여 리스트로 반환합니다

In [ ]:
import os

# OpenAI API 키 직접 설정
os.environ["OPENAI_API_KEY"] = 'YOUR_OPENAI_API_KEY'

In [ ]:
!pip install pypdf

In [ ]:
from langchain.text_splitter import CharacterTextSplitter  # 텍스트 분할을 위해 사용
import re
import json

def extract_and_parse_json(text):
    """
    텍스트에서 JSON 문자열을 추출하여 사전형으로 변환하는 기능.
    JSON 문자열이 잘못된 형식이거나 포함된 경우 오류 처리.
    """
    try:
        # 응답에서 JSON을 바로 파싱해보는 시도
        return json.loads(text)
    except json.JSONDecodeError:
        # 만약 JSON 변환에 실패할 경우 정규 표현식이나 다른 방식으로 추출 시도
        print("JSON 파싱 실패: 텍스트를 정규화하여 다시 시도합니다.")
        try:
            # 텍스트에서 JSON 부분만 추출하기 위한 정규 표현식 처리
            match = re.search(r"\[.*\]", text, re.DOTALL)
            json_string = match.group() if match else ""
            return json.loads(json_string)
        except (AttributeError, json.JSONDecodeError) as e:
            print(f"JSON 변환 오류: {e}")
            return {}

def load_pdf_to_json(pdf_path):
    """
    단일 PDF 파일을 읽어 JSON 형식으로 변환하는 함수
    """
    import os
    from langchain_community.chat_models import ChatOpenAI
    from langchain.schema import HumanMessage

    openai_api_key = os.environ.get("OPENAI_API_KEY")

    if not openai_api_key:
        raise ValueError("OpenAI API 키가 설정되지 않았습니다. OPENAI_API_KEY 환경 변수를 설정하세요.")

    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)

    # PyPDFLoader를 사용해 PDF 파일 읽기
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    contents = []

    # 페이지 텍스트를 하나의 긴 텍스트로 결합
    full_text = "".join([page.page_content for page in pages])

    # 텍스트를 1000자 단위로 나누고, 각 조각에서 200자씩 중복되도록 설정
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(full_text)

    # 각 분할된 텍스트에 대해 질문-답변 형식으로 변환
    for chunk in chunks:
        prompt = f"""
        당신은 건물 보안요원을 위한 매뉴얼 챗봇입니다.
        건물 보안 매뉴얼은 총 세부 수칙 14개로 구성되어 있습니다.
        각각의 세부 수칙에 따라 다양한 상황에 대한 질문을 만들고, 매뉴얼을 바탕으로 하는 답변을 생성하세요.
        매뉴얼 내의 모든 세부 수칙을 반영하여 질문과 답변을 작성하십시오.
        매뉴얼을 기반으로하여 최소 36개의 질문- 답변 쌍을 생성해야 합니다.

        ### 형식:
        [
            {{
                "input": "질문 내용",
                "output": "답변 내용"
            }},
            ...
        ]

        반드시 JSON 형식으로만 출력하고, 다른 텍스트는 포함하지 마세요.

        텍스트:
        {chunk}
        """

        try:
            # 모델 호출
            result = llm.invoke([HumanMessage(content=prompt)])

            # 모델 응답 출력 (디버깅용)
            print(f"Model response for chunk:\n{result.content}\n")

            # JSON 변환 시도
            json_data = extract_and_parse_json(result.content)
            if json_data:
                contents.append(json_data)
            else:
                print(f"Warning: No valid JSON extracted from this chunk.\n")

        except Exception as e:
            print(f"Error processing chunk: {e}\n")

    return contents

# PDF 경로 설정
pdf_path = "/content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/건물 보안 매뉴얼.pdf"

# PDF 파일을 JSON 데이터로 변환
json_data = load_pdf_to_json(pdf_path)

# 변환된 JSON 데이터 확인
print(json_data)

<ipython-input-13-f60098272ad0>:38: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0, openai_api_key=openai_api_key)


Model response for chunk:
[
    {
        "input": "근무 시간을 늦추거나 앞당기는 것을 방지하기 위한 규칙은 무엇인가요?",
        "output": "출퇴근 및 교대 시간을 준수하여 늦거나 빠른 출퇴근을 방지합니다."
    },
    {
        "input": "보안 요원이 항상 소지해야 하는 보안 장비는 무엇인가요?",
        "output": "제복 착용 및 보안 장비인 무전기, ID카드, 손전등을 항상 소지해야 합니다."
    },
    {
        "input": "근무 중에는 어떤 태도를 유지해야 하나요?",
        "output": "항상 경계 태세를 유지하고, 불필요한 잡담 및 개인 스마트폰 사용을 금지해야 합니다."
    },
    {
        "input": "출입자 탐지 시스템은 무엇을 실시간으로 탐지하나요?",
        "output": "출입자 탐지 시스템은 보안 시설에 접근하는 모든 인원을 실시간으로 탐지합니다."
    },
    {
        "input": "출입자의 출입 기록을 확인하기 위해 무엇을 확인해야 하나요?",
        "output": "출입자는 출입 기록 작성 후 신분증 확인을 해야 합니다."
    },
    {
        "input": "CCTV 모니터링 시 이상 상황이 발생했을 때 어떻게 대응해야 하나요?",
        "output": "이상 상황 발생 시 즉각 대응하여 상황을 처리해야 합니다."
    },
    {
        "input": "화재 발생 시 저층부에서는 어떤 대피 경로를 이용해야 하나요?",
        "output": "저층부는 대피 경로가 짧으므로 비상구를 통해 신속하게 외부로 대피해야 합니다."
    },
    {
        "input": "화재 발생 시 고층부에서는 어떤 대피 우선 순위를 가져야 하나요?",
        "output": "고층부는

## 3단계. JSON 파일로 저장

In [ ]:
import json
def save_json_to_file(json_data, json_file_path):
    """
    변환된 JSON 데이터를 파일로 저장하는 함수
    """
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, ensure_ascii=False, indent=4)

# JSON 파일 경로 설정
json_file_path = "/content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/output.json"

# JSON 데이터를 파일로 저장
save_json_to_file(json_data, json_file_path)

## 4단계. JSON 파일을 Hugging Face 형식으로 변환

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import Dataset
import json
import os

def convert_to_huggingface_format(json_file_path, output_path):
    """
    JSON 파일을 Hugging Face Dataset 형식으로 변환하는 함수
    """
    indataset = []

    try:
        # JSON 파일이 있는지 확인
        if not os.path.exists(json_file_path):
            print(f"Error: JSON 파일이 존재하지 않습니다. 경로: {json_file_path}")
            return

        # JSON 파일을 열고 데이터 파싱
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

            print(f"데이터의 길이: {len(data)}")

            # 데이터를 순회하며 변환
            for item in data:
                question = item.get("input", "").strip()
                answer = item.get("output", "").strip()

                # 데이터 무결성 확인
                if question and answer:
                    indataset.append(f"Q: {question} -> A: {answer}")

        # 변환된 데이터셋을 Hugging Face Dataset 형식으로 변환
        dataset = Dataset.from_dict({'text': indataset})

        # 데이터셋을 지정된 경로에 저장
        dataset.save_to_disk(output_path)

        print(f"데이터셋이 성공적으로 변환되고 저장되었습니다. 경로: {output_path}")
        return dataset

    except Exception as e:
        print(f"Error during converting: {e}")

# 파일 경로 설정
json_file_path = '/content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/new2.json'
output_path = '/content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/hf_dataset2'

# 함수 실행
convert_to_huggingface_format(json_file_path, output_path)

데이터의 길이: 70


Saving the dataset (0/1 shards):   0%|          | 0/70 [00:00<?, ? examples/s]

데이터셋이 성공적으로 변환되고 저장되었습니다. 경로: /content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/hf_dataset2


Dataset({
    features: ['text'],
    num_rows: 70
})

## 5단계. Hugging Face에 데이터셋 업로드

In [ ]:
import huggingface_hub
huggingface_hub.login()

In [ ]:
import huggingface_hub
from datasets import Dataset

# 데이터셋 로드
dataset = Dataset.load_from_disk('/content/drive/MyDrive/Colab Notebooks/Finetuned/dataset/hf_dataset2')

# 데이터셋 정보 확인
print("데이터셋 정보 확인:")
print(dataset)

# 4단계. 허깅페이스에 Dataset 업로드
dataset.push_to_hub("kingkim/DS_Building_SecurityManual_V3")

print("데이터셋이 Hugging Face에 성공적으로 업로드되었습니다.")

데이터셋 정보 확인:
Dataset({
    features: ['text'],
    num_rows: 70
})


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

데이터셋이 Hugging Face에 성공적으로 업로드되었습니다.
